# Asignación de turnos de los agentes de tránsito usando el modelo


En este notebook, se usa el modelo para asignar los turnos que los agentes de tránsito deberían patrullar cada día, buscando cubrir la mayor cantidad de accidentes posibles (un accidente se considera cubierto si ocurre en un barrio y a una fecha y hora en la que había un agente asignando patrullando dicho turno). La cantidad de accidentes cubiertos al asignar los agentes con el modelo se compara con los que se hubieran cubierto al asignar los agentes en otros escenarios: una asignación de turnos completamente aleatoria, asignar barrios con más accidentes en horas de la mañana (8, 9, 10 y 11am) y barrios con más accidentes en horas de la tarde (2, 3, 4 y 5pm). Se considera que los agentes pueden patrullar 4 horas diarias. Evaluamos varios posibles números de agentes disponibles para patrullar para poder observar el comportamiento del modelo en varios escenarios.

In [1]:
### Realizamos el cambio de directorio de trabajo al "Directorio Base"
import os
current_dir = os.getcwd()
base_path = os.path.dirname(current_dir)


os.chdir(base_path)

import numpy as np
import random
import pandas as pd
import datetime as dt
from random import sample 
from sklearn.metrics import precision_score
import os
import sys

random.seed(42)

In [2]:
### Lee datos de train (para hallar barrios con mas accidentes en train)
data_train_u = pd.read_csv('data/train.csv') 
data_val = pd.read_csv('data/validation.csv') 
data_train = pd.concat([data_train_u, data_val])



### Lee datos de test
data_test_completa = pd.read_csv('data/test.csv')
data_test = data_test_completa.drop(['BARRIO', 'Accidente', 'TW'], axis=1)


In [3]:
### Modelo usado
sys.path.insert(0, os.getcwd())
import scripts.funciones as funciones
mod_version = funciones.carga_model('.', f'models/verFinal', 'verFinal_voting')
mod = mod_version['model'].steps[0][1]

classifier = mod_version['model'].steps[1][1]

C:\Users\nicol\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
C:\Users\nicol\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\nicol\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as

In [4]:
### Calcular predicciones en test
predicciones = classifier.predict(data_test)
probabilidades = classifier.predict_proba(data_test)[:,1]

In [5]:
#### Consolidar data_test
data_test_full = data_test_completa.copy()
data_test_full['predic_modelo'] = probabilidades

data_test_full['TW'] = pd.to_datetime(data_test_full['TW'])
data_test_full['hour'] = data_test_full['TW'].dt.hour

#### Ordenar por fecha
data_test_full = data_test_full.sort_values(by=['TW','BARRIO']).reset_index(drop=True)
data_test_full = data_test_full[['TW','Accidente', 'predic_modelo','BARRIO']]


#### Evaluar varios escenarios
A partir de aquí, evaluaremos el rendimiento de la asignación del modelo y de las demás asignaciones en varios escenarios, lo que varía entre los escenarios es el número de agentes diarios que se pueden asignar.
Se imprime la cantidad de accidentes cubiertos con cada criterio de asignación y en cada escenario. Esta cantidad de accidentes cubiertos es la que usamos para construir luego la tabla que presentamos en el informe que compara los distintos criterios de asignación en los distintos escenarios.


In [6]:


### Se asume que cada agente cubre x turnos diarios
turnos_diarios_agente = 4


### Se varia el numero de agentes disponibles entre los escenarios
for num_agentes in range(3,9):

    ### Numero de turnos totales diarios que se cubren
    agentes = num_agentes * turnos_diarios_agente  

    #### Numero de Barrios de la comuna EL Poblado
    numb = len(pd.unique(data_test_full['BARRIO']))


    ### Los barrios con mas choques en el conjunto de entrenamiento son
    accis = data_train.groupby('BARRIO').sum()['Accidente']
    most_accis = np.argsort(accis.values)[::-1]
    barrs = accis.index[most_accis[:num_agentes]]

   

    ### Accidentes reales
    acci_reales = data_test_full['Accidente']






    ######### Ciclo para empezar a comparar
    numb = numb*24 #Numero de turnos a asignar en cada dia
    
    dias_tota = len(data_test_full)/numb  ### total de dias

    aci1b = []
    aci2 = []
    aci3 = []
    
    ### Esta variable es solo interna para analisis (para ir viendo donde se asignaron)
    dic_toto = {}
    for i in range(24):
        dic_toto[i] = 1
        
    ### Asignacion de cada dia
    for dia in range(int(dias_tota)):

        ### Caso 1: prediccion modelo
        
        #### Las probabilidades de accidentalidad para todas las observaciones de este dia
        acci_prob = data_test_full['predic_modelo'][dia*numb:(dia+1)*numb].values

        
        ### Este bloque sirve para asignar a los agentes a las horas y barrios con mayor probabilidad
        ### Se ordenan de forma descendente todas las probabilidades de todas las horas y barrios de cada dia,
        ### y se van asignando desde la observacion uno (la de mayor probabilidad) en adelante
        ### Se verifica que no se asignen a la misma hora un numero de agentes mayor a los disponibles
        max_por_hora = num_agentes
        acci_pre = data_test_full[dia*numb:(dia+1)*numb].reset_index(drop=True).sort_values(by='predic_modelo', ascending=False)
        new_accip = []
        dic_cumul = {}
        for di in np.argsort(acci_prob)[::-1]:
            esta_o = acci_pre.loc[di]
            if esta_o[0].hour not in dic_cumul:
                new_accip.append(di)
                dic_cumul[esta_o[0].hour] = 1
                dic_toto[esta_o[0].hour] = dic_toto[esta_o[0].hour] + 1
            else:
                if dic_cumul[esta_o[0].hour]<max_por_hora:  ### Maximo, asignar num_agentes turnos en cada hora
                    new_accip.append(di)
                    dic_cumul[esta_o[0].hour] = dic_cumul[esta_o[0].hour] + 1
                    dic_toto[esta_o[0].hour] = dic_toto[esta_o[0].hour] + 1
            if len(new_accip)==agentes:
                break
        predigo1b = np.zeros(numb)
        predigo1b[new_accip]=1
        aci1b.extend(list(predigo1b))


        
        
        ### Caso 2: Asignacion completamente aleatoria cada dia
        predigo2 = np.zeros(numb)
        accialeat = sample(range(numb),agentes)    
        predigo2[accialeat]=1
        aci2.extend(list(predigo2))



    ### Caso 3: Barrios con mas accidentes, horas mañana
    numbar = len(pd.unique(data_test_full['BARRIO']))
    horas_tota = len(data_test_full)/numbar
    horas_pico = [8,9,10,11]
    aci3 = []
    for hor in range(int(horas_tota)):
        predigo3 = np.zeros(numbar)
        if hor%24 in horas_pico:
            predigo3[most_accis[:num_agentes]]=1
        aci3.extend(list(predigo3))





    ### Caso 4: Barrios con mas accidentes, horas tarde
    numbar = len(pd.unique(data_test_full['BARRIO']))
    horas_tota = len(data_test_full)/numbar
    horas_pico = [14,15,16,17]
    aci4 = []
    for hor in range(int(horas_tota)):
        predigo4 = np.zeros(numbar)
        if hor%24 in horas_pico:
            predigo4[most_accis[:num_agentes]]=1
        aci4.extend(list(predigo4))


        
        
    print('\n\n\n\nAgentes patrullando: '+str(num_agentes))
    print('\nLos barrios con mas accidentes son: ')
    print(list(barrs))


    
    
    ###  Asignacion del modelo
    precision =precision_score(acci_reales.values, aci1b)
    print('\nAsignacion modelo. ')
    print(f'Se cubrieron {round(precision*agentes*dias_tota)} accidentes.')


    ### Aleatorio
    precision =precision_score(acci_reales.values, aci2)
    print('\nAsignacion aleatoria. ')
    print(f'Se cubrieron {round(precision*agentes*dias_tota)} accidentes.')

    


    ### Barrio con mas accidentes, horas de la manana
    precision =precision_score(acci_reales.values, aci3)
    print('\nAsignacion a barrios con mas accidentes (horas mañana). ')
    print(f'Se cubrieron {round(precision*agentes*dias_tota)} accidentes.')

    ### Barrios con mas accidentes, horas de la tarde
    precision =precision_score(acci_reales.values, aci4)
    print('\nAsignacion a barrios con mas accidentes (horas tarde). ')
    print(f'Se cubrieron {round(precision*agentes*dias_tota)} accidentes.')






Agentes patrullando: 3

Los barrios con mas accidentes son: 
['villacarlota', 'laaguacatala', 'manila']

Asignacion modelo. 
Se cubrieron 138.0 accidentes.

Asignacion aleatoria. 
Se cubrieron 37.0 accidentes.

Asignacion a barrios con mas accidentes (horas mañana). 
Se cubrieron 129.0 accidentes.

Asignacion a barrios con mas accidentes (horas tarde). 
Se cubrieron 132.0 accidentes.




Agentes patrullando: 4

Los barrios con mas accidentes son: 
['villacarlota', 'laaguacatala', 'manila', 'elpoblado']

Asignacion modelo. 
Se cubrieron 166.0 accidentes.

Asignacion aleatoria. 
Se cubrieron 44.0 accidentes.

Asignacion a barrios con mas accidentes (horas mañana). 
Se cubrieron 159.0 accidentes.

Asignacion a barrios con mas accidentes (horas tarde). 
Se cubrieron 172.0 accidentes.




Agentes patrullando: 5

Los barrios con mas accidentes son: 
['villacarlota', 'laaguacatala', 'manila', 'elpoblado', 'barriocolombia']

Asignacion modelo. 
Se cubrieron 222.0 accidentes.

Asignacion al